In [19]:
#Import Necessary Packages
import numpy as np
import pandas as pd

#Import Dataset
data = pd.read_csv('fifa21_training.csv')
data = data.drop(['Nationality', 'Club'], axis = 1)
data.head()

,Name,Age,Default Position,Height,Weight,Value,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,AW,DW,IR,PAC,SHO,PAS,DRI,DEF,PHY,OVA
0,A. Pasche,26,CM,"5'9""",161lbs,€525K,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,64
1,Alan Carvalho,30,ST,"6'0""",159lbs,€8.5M,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,77
2,S. Giovinco,33,CAM,"5'4""",134lbs,€9M,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4 ★,4★,High,Medium,2 ★,80,77,78,86,27,56,80
3,J. Evans,22,CDM,"5'10""",152lbs,€275K,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2 ★,2★,Medium,Medium,1 ★,57,44,54,57,57,60,59
4,Y. Demoncy,23,CDM,"5'11""",150lbs,€725K,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2 ★,3★,Low,Medium,1 ★,66,44,60,64,60,66,65


In [20]:
#Checking dtypes prior to cleaning
pd.set_option('display.max_rows', 120)
pd.set_option('display.max_columns', 70)
data.dtypes

Name                 object
Age                   int64
Default Position     object
Height               object
Weight               object
Value                object
Attacking             int64
Crossing              int64
Finishing             int64
Heading Accuracy      int64
Short Passing         int64
Volleys             float64
Skill                 int64
Dribbling             int64
Curve               float64
FK Accuracy           int64
Long Passing          int64
Ball Control          int64
Movement              int64
Acceleration          int64
Sprint Speed          int64
Agility             float64
Reactions             int64
Balance             float64
Power                 int64
Shot Power            int64
Jumping             float64
Stamina               int64
Strength              int64
Long Shots            int64
Mentality             int64
Aggression            int64
Interceptions       float64
Positioning         float64
Vision              float64
Penalties           

In [21]:
#Define a Function for Cleaning 
def clean(x):
    #Drop Rows with Null Values - In this Dataset it is only inactive players, a small sample
    data.dropna(inplace=True)
    
    #Stripping " from the Height row so that it can be split on '
    data['Height'] = data['Height'].str.strip("\"")
    
    #Creating a New Dataframe with the split columns
    new = data['Height'].str.split("\'", n = 1, expand = True)
    
    #Define Foot > Inches Conversion
    def ftin(x):
        return int (x)*12
    
    #Apply the function to the new column
    data['Inches2'] = list(map(ftin, new[0]))
    data["Inches"]= new[1]
    data['Height'] = data[['Inches', 'Inches2']].sum(axis = 1)
    
    #Define Inches to CM Conversion
    def toCM(x):
        return float (x)*2.54
    
    #Convert Corrected Heights into CM
    data['Height'] = list(map(toCM, data['Height']))
    
    #Drop Columns used for Manipulation
    data.drop(['Inches', 'Inches2'], axis=1, inplace=True)
    

In [22]:
clean(data)


In [23]:
data.head()

,Name,Age,Default Position,Height,Weight,Value,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,AW,DW,IR,PAC,SHO,PAS,DRI,DEF,PHY,OVA
0,A. Pasche,26,CM,152.40,161lbs,€525K,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,64
1,Alan Carvalho,30,ST,182.88,159lbs,€8.5M,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,77
2,S. Giovinco,33,CAM,152.40,134lbs,€9M,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4 ★,4★,High,Medium,2 ★,80,77,78,86,27,56,80
3,J. Evans,22,CDM,152.40,152lbs,€275K,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2 ★,2★,Medium,Medium,1 ★,57,44,54,57,57,60,59
4,Y. Demoncy,23,CDM,152.40,150lbs,€725K,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2 ★,3★,Low,Medium,1 ★,66,44,60,64,60,66,65


In [ ]:
#attacking, skill, movement, power, mentality, defending
#remove goalkeepers

In [ ]:
X_num = data.select_dtypes(include = np.number)
X_cat = data.select_dtypes(include = np.object)

In [ ]:
manip = X_cat.drop(['Name', 'Height','Weight', 'Value', 'W/F', 'SM', 'IR'], axis = 1)

In [ ]:
manip.shape

In [ ]:
#Encoding the Dummies using OneHot
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='error', drop='first').fit(manip)
encoded = encoder.transform(manip).toarray()
encoded

In [ ]:
X_num.head()

In [ ]:
#Define a Function for Cleaning 
def clean(x):
    #Drop Rows with Null Values - In this Dataset it is only inactive players, a small sample
    data.dropna(inplace=True)
    
    #Stripping " from the Height row so that it can be split on '
    data['Height'] = data['Height'].str.strip("\"")
    
    #Creating a New Dataframe with the split columns
    new = data['Height'].str.split("\'", n = 1, expand = True)
    
    #Define Foot > Inches Conversion
    def ftin(x):
        return int (x)*12
    #Create New Columns with split Inch Values
    data['Inches2'] = list(map(ftin, new[0]))
    data["Inches"]= new[1]
    
    #Replace Heights with I
    data['Height'] = data[['Inches', 'Inches2']].sum(axis = 1)
    
    #Define Inches to CM Conversion
    def toCM(x):
        return float (x)*2.54